## Assignement 2: Exploritory Data Analysis

William "Taylor" Martinez

#### Notes and Setup

[`dplyr` to `polars`](https://docs.pola.rs/user-guide/migration/pandas/#column-assignment):
| Operation                | Syntax                                   |
|--------------------------|------------------------------------------|
| read (lazy)              | `df.scan_csv()` or `df.scan_parquet()`  |
| collect                  | `df.collect()`                           |
| select                   | `df.select("col_name1", "col_name2")`   |
| filter                   | `df.filter(pl.col("col_name") < 10)`    |
| mutate                   | `df.with_columns(new_col_name = pl.col("col_name") * 10)` |
| mutate (conditional)   | ```df.with_columns( pl.when(pl.col("c") == 2) .then(pl.col("b")) .otherwise(pl.col("a")).alias("a") )``` |
| missing                  | `null`                                   |
| all columns               | `pl.all()`                                  |

`csv` vs `parquet`:
    Parquet was chosen over `csv` because it takes up less space, it is columnar formatted, and is has improved query performance. [medium](https://medium.com/@dinesh1.chopra/unveiling-the-battle-apache-parquet-vs-csv-exploring-the-pros-and-cons-of-data-formats-b6bfd8e43107)








In [1]:
import polars as pl

In [2]:
# Convert to parquet and read in the data
data_path = "../../Data/"
df = pl.scan_csv(data_path + "heart_2022_with_nans.csv")
df.collect().write_parquet(data_path + "heart_2022_with_nans.parquet")
df = pl.scan_parquet(data_path + "heart_2022_with_nans.parquet")

df.fetch(5)

### EDA Task 1: Create `HadHeartDisease` column

1. Set `HadHeartDisease` to `True` if the survey participant reported having a least one of the following adverse cardiovascular events: heart attack (`HadHeartAttack`), stroke (`HadStroke`), or angina  (`HadAngina`).

In [77]:
df = df.with_columns(
    pl.when(
        (pl.col("HadHeartAttack") == "Yes") |
        (pl.col("HadStroke") == "Yes") |
        (pl.col("HadAngina") == "Yes")
    )
    .then(pl.lit("Yes"))
    .otherwise(pl.lit("No"))
    .alias("HadHeartDisease")
)

df.fetch(5)

State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadDiabetes,DeafOrHardOfHearing,BlindOrVisionDifficulty,DifficultyConcentrating,DifficultyWalking,DifficultyDressingBathing,DifficultyErrands,SmokerStatus,ECigaretteUsage,ChestScan,RaceEthnicityCategory,AgeCategory,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos,HadHeartDisease
str,str,str,f64,f64,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,str,str,str
"""Alabama""","""Female""","""Very good""",0.0,0.0,"""Within past ye…","""No""",8.0,null,"""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""No""","""Never smoked""","""Not at all (ri…","""No""","""White only, No…","""Age 80 or olde…",null,null,null,"""No""","""No""","""Yes""","""No""","""Yes, received …","""No""","""No""","""No"""
"""Alabama""","""Female""","""Excellent""",0.0,0.0,null,"""No""",6.0,null,"""No""","""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Never smoked""","""Never used e-c…","""No""","""White only, No…","""Age 80 or olde…",1.6,68.04,26.57,"""No""","""No""","""No""","""No""","""No, did not re…","""No""","""No""","""No"""
"""Alabama""","""Female""","""Very good""",2.0,3.0,"""Within past ye…","""Yes""",5.0,null,"""No""","""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Never smoked""","""Never used e-c…","""No""","""White only, No…","""Age 55 to 59""",1.57,63.5,25.61,"""No""","""No""","""No""","""No""",null,"""No""","""Yes""","""No"""
"""Alabama""","""Female""","""Excellent""",0.0,0.0,"""Within past ye…","""Yes""",7.0,null,"""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Current smoker…","""Never used e-c…","""Yes""","""White only, No…",null,1.65,63.5,23.3,"""No""","""No""","""Yes""","""Yes""","""No, did not re…","""No""","""No""","""No"""
"""Alabama""","""Female""","""Fair""",2.0,0.0,"""Within past ye…","""Yes""",9.0,null,"""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Never smoked""","""Never used e-c…","""Yes""","""White only, No…","""Age 40 to 44""",1.57,53.98,21.77,"""Yes""","""No""","""No""","""Yes""","""No, did not re…","""No""","""No""","""No"""


### EDA Task 2: Drop Observations With Too Many Missing Values

1. Create `df_heart_drop` where participants are dropped if Heart attack (`HadHeartAttack`), stroke (`HadStroke`), or angina  (`HadAngina`) are missing.

2. From `df_heart_drop`, make multiple dataframes that drop survey participants based
on the number of missing responses.

3. Collect the dataframes and return the length of each entry.


In [70]:
df_heart_drop = df.drop_nulls(subset=["HadHeartAttack", "HadStroke", "HadAngina"])


State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadDiabetes,DeafOrHardOfHearing,BlindOrVisionDifficulty,DifficultyConcentrating,DifficultyWalking,DifficultyDressingBathing,DifficultyErrands,SmokerStatus,ECigaretteUsage,ChestScan,RaceEthnicityCategory,AgeCategory,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos,HadHeartDisease
str,str,str,f64,f64,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,str,str,str
"""Alabama""","""Female""","""Very good""",0.0,0.0,"""Within past ye…","""No""",8.0,null,"""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""No""","""Never smoked""","""Not at all (ri…","""No""","""White only, No…","""Age 80 or olde…",null,null,null,"""No""","""No""","""Yes""","""No""","""Yes, received …","""No""","""No""","""No"""
"""Alabama""","""Female""","""Excellent""",0.0,0.0,null,"""No""",6.0,null,"""No""","""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Never smoked""","""Never used e-c…","""No""","""White only, No…","""Age 80 or olde…",1.6,68.04,26.57,"""No""","""No""","""No""","""No""","""No, did not re…","""No""","""No""","""No"""
"""Alabama""","""Female""","""Very good""",2.0,3.0,"""Within past ye…","""Yes""",5.0,null,"""No""","""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Never smoked""","""Never used e-c…","""No""","""White only, No…","""Age 55 to 59""",1.57,63.5,25.61,"""No""","""No""","""No""","""No""",null,"""No""","""Yes""","""No"""
"""Alabama""","""Female""","""Excellent""",0.0,0.0,"""Within past ye…","""Yes""",7.0,null,"""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Current smoker…","""Never used e-c…","""Yes""","""White only, No…",null,1.65,63.5,23.3,"""No""","""No""","""Yes""","""Yes""","""No, did not re…","""No""","""No""","""No"""
"""Alabama""","""Female""","""Fair""",2.0,0.0,"""Within past ye…","""Yes""",9.0,null,"""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Never smoked""","""Never used e-c…","""Yes""","""White only, No…","""Age 40 to 44""",1.57,53.98,21.77,"""Yes""","""No""","""No""","""Yes""","""No, did not re…","""No""","""No""","""No"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Alabama""","""Female""","""Excellent""",0.0,0.0,"""Within past ye…","""Yes""",8.0,null,"""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""",null,"""Never used e-c…","""No""","""White only, No…","""Age 65 to 69""",1.57,64.41,25.97,"""No""",null,"""Yes""","""No""","""Yes, received …","""No""","""No""","""No"""
"""Alabama""","""Female""","""Excellent""",0.0,0.0,"""Within past ye…","""Yes""",6.0,null,"""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Never smoked""","""Never used e-c…","""Yes""","""Black only, No…",null,1.57,61.23,24.69,"""No""","""No""","""Yes""","""Yes""","""Yes, received …","""No""","""No""","""No"""
"""Alabama""","""Female""","""Fair""",0.0,15.0,"""Within past ye…","""Yes""",6.0,null,"""No""","""No""","""No""","""Yes""","""Yes""","""Yes""","""No""","""No""","""Yes""","""Yes""","""No""","""No""","""No""","""Yes""","""No""","""Yes""","""Former smoker""","""Never used e

In [76]:
### Automatically drop observations where HadHeartAttack, HadStroke, and HadAngina are missing.
df_heart_drop = df.drop_nulls(subset=["HadHeartAttack", "HadStroke", "HadAngina"])

### Drop Observations Based On Number Of Missing Values
thresholds = [0, 1, 3, 5, 10, 20, 40] # if number of missing is > threshold, drop the observation.
for threshold in thresholds:
    df_name = f"df_heart_drop_{threshold:02}" 
    # Drop if the row sum of all columns where the value is null is greater than threshold
    globals()[df_name] = df_heart_drop.filter(pl.sum_horizontal(pl.all().is_null()) <= threshold)
    # Print the number of rows in each dataframe.
    print(df_name + ": " + str(globals()[df_name].select(pl.len()).collect().item()))

df_heart_drop_00: 246022
df_heart_drop_01: 331181
df_heart_drop_03: 381718
df_heart_drop_05: 391725
df_heart_drop_10: 410245
df_heart_drop_20: 436507
df_heart_drop_40: 437510


### EDA Task 3: Impute remaining missing values

1. Show column types

2. Impute float and integer values by median.

3. Impute string values by mode.

Note: This is applied to df_heart_drop, other dataframes can be imputed.


{String, Float64}


In [62]:
### Unique data types:
# print(set(df_heart_drop.dtypes))

### Impute 
def impute_df(df):
    df_col = df.collect()
    for i in range(len(df_col.columns)):
        col_name = df_col.columns[i]
        dtype = df_col.dtypes[i]

        if dtype == pl.Utf8:
            mode_series = df_col[col_name].mode()
            if not mode_series.is_empty(): 
                mode_value = mode_series[0] 
                df_col = df_col.with_columns(
                    df_col[col_name].fill_null(mode_value)
                )
        elif dtype == pl.Float64:
            median_value = df_col[col_name].median()
            if median_value is not None:
                df_col = df_col.with_columns(
                    df_col[col_name].fill_null(median_value)
                )
        else:
            print("Unexpected type:", dtype)
    return df_col

impute_df(df_heart_drop).null_count()


State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadDiabetes,DeafOrHardOfHearing,BlindOrVisionDifficulty,DifficultyConcentrating,DifficultyWalking,DifficultyDressingBathing,DifficultyErrands,SmokerStatus,ECigaretteUsage,ChestScan,RaceEthnicityCategory,AgeCategory,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos,HadHeartDisease
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### EDA Task 4: Show tables/graphs representing each task

1. Table comparing the number of observations of `HadHeartAttack`, `HadStroke`, `HadAngina` and the summarized column `HadHeartDisease`.
    a. Provide an example using a rudimentary model (Logit Regression with enet regularization)

2. Table showing the number of observations after setting a missing threshold.
    a. Provide an example using a rudimentary model (Logit Regression with enet regularization)

3. Table comparing using imputation.

In [ ]:
# 